In [9]:
from enum import Enum
import csv
import json
import datetime
# parse json file
FILE = "scoreboard.json"

with open(FILE) as f:
    data = json.load(f)
data = data

class Status(Enum):
    OK = 101
    CORRUPT = 102
    MUMBLE = 103
    DOWN = 104
    CHECKER_ERROR = 110
    UNINITIALIZED = 111

class Service(Enum):
    TRAINING = 0
    PASSMGR = 1
    PLACES = 2
    LOCKSTONE = 3
    GODEEPER = 4
    PURE = 5
    FUNDING = 6
    KEYS = 7
    SPACES = 8
    DOCS = 9
    TOKENOURCER = 10
    NOTES = 11

class Colors(Enum):
    DEFAULT = "\033[38;2;212;253;255m"
    GOLD = "\033[38;2;201;176;55m"
    SILVER = "\033[38;2;215;215;215m"
    BRONZE = "\033[38;2;173;138;86m"

    RED = "\033[38;2;255;0;0m"
    GREEN = "\033[38;2;0;255;0m"
    BLUE = "\033[38;2;0;0;255m"
    BLACK = "\033[38;2;0;0;0m"
    WHITE = "\033[38;2;255;255;255m"
    ORANGE = "\033[38;2;255;165;0m"
    PURPLE = "\033[38;2;128;0;128m"
    YELLOW = "\033[38;2;255;255;0m"
    PINK = "\033[38;2;255;192;203m"
    CYAN = "\033[38;2;0;255;255m"
    GRAY = "\033[38;2;128;128;128m"
    DARK_GRAY = "\033[38;2;64;64;64m"
    LIGHT_RED = "\033[38;2;255;102;102m"
    DARK_RED = "\033[38;2;139;0;0m"
    LIGHT_GREEN = "\033[38;2;102;255;102m"
    DARK_GREEN = "\033[38;2;0;100;0m"
    LIGHT_BLUE = "\033[38;2;102;102;255m"
    DARK_BLUE = "\033[38;2;0;0;139m"

    BOLD = "\033[1m"
    ITALIC = "\033[3m"
    UNDERLINE = "\033[4m"
    STRIKETHROUGH = "\033[9m"

    BG_BLACK = "\033[40m"
    BG_RED = "\033[41m"
    BG_GREEN = "\033[42m"
    BG_YELLOW = "\033[43m"
    BG_BLUE = "\033[44m"
    BG_MAGENTA = "\033[45m"
    BG_CYAN = "\033[46m"
    BG_WHITE = "\033[47m"

    RESET = "\033[0m"

TOTAL_ROUNDS = len(data)
TOTAL_TEAMS = len(data[0]['scoreboard'])

TEAM_ID = 3

WINNERS = [125, 69, 81]

def get_service_name(service: Enum):
    return f"{Colors.PURPLE.value}{Colors.BOLD.value}{service.name}{Colors.RESET.value}"

def get_round_number(round_number: int, color = Colors.LIGHT_BLUE):
    return f"{color.value}{round_number}{Colors.RESET.value}"

def get_service_start_round(service: Enum, data: list):
    # Set up the initial state
    for round in range(TOTAL_ROUNDS):
        if data[round]['scoreboard'][0]['services'][service.value]['status'] == Status.OK.value:
            return round

def compute_service_price_for_round(service: int, round_number: int, data: list):
    # Set up the initial state
    if round_number == 0:
        return False
    if round_number == 1:
        return 10.0
    
    round_data = data[round_number - 1]
    prev_round_data = data[round_number - 2]

    prices_for_round = []
    for idx, team in enumerate(round_data['scoreboard']):
        service_data = team['services'][service]

        prev_flags = prev_round_data['scoreboard'][idx]['services'][service]['flags']
        prev_sflags = prev_round_data['scoreboard'][idx]['services'][service]['sflags']
        prev_fp = prev_round_data['scoreboard'][idx]['services'][service]['fp']

        current_flags = service_data['flags']
        current_sflags = service_data['sflags']
        current_fp = service_data['fp']

        flags_diff = current_flags - prev_flags
        sflags_diff = current_sflags - prev_sflags
        fp_diff = current_fp - prev_fp

        net_flags_gained = flags_diff - sflags_diff

        # If there are net flags gained, compute the price
        if net_flags_gained > 0:
            price = fp_diff / net_flags_gained
            prices_for_round.append(price)


    # Compute and return the average flag price for the round
    if prices_for_round:  # Ensure the list is not empty
        avg_price = sum(prices_for_round) / len(prices_for_round)
        return avg_price
    else:
        # compute average price for the previous round and next round
        prev_avg_price = compute_service_price_for_round(service, round_number - 1, data)
        next_avg_price = compute_service_price_for_round(service, round_number + 1, data)
        if prev_avg_price and next_avg_price:
            return (prev_avg_price + next_avg_price) / 2
        else:
            return "No net flags gained for the service in this round."

def get_team_name(team_id: int, coloring : bool = True) -> str:
    # if BOT team
    if team_id > 219:
        return f"{Colors.LIGHT_GREEN.value}BOT Team #{team_id - 219}{Colors.RESET.value}"
    
    with open("teams.csv") as f:
        reader = csv.reader(f)
        for row in reader:
            if int(row[0]) == team_id:
                if not coloring:
                    return row[1]
                COLOR = Colors.DEFAULT.value
                if team_id in WINNERS:
                    if team_id == WINNERS[0]:
                        COLOR = Colors.GOLD.value + Colors.BOLD.value
                    elif team_id == WINNERS[1]:
                        COLOR = Colors.SILVER.value + Colors.BOLD.value
                    elif team_id == WINNERS[2]:
                        COLOR = Colors.BRONZE.value + Colors.BOLD.value
                elif team_id == TEAM_ID:
                    COLOR = Colors.LIGHT_RED.value + Colors.BOLD.value
                return COLOR + row[1] + Colors.RESET.value
        return "Team not found!"

def first_flag(service: Enum, data):
    # Iterate over each round in the data
    for round, round_data in enumerate(data):
        # Iterate over each team in the round's scoreboard
        target_team = None
        source_team = None

        for team in round_data['scoreboard']:
            service_data = team['services'][service.value]
            # If stolen flags (sflags) exist for this service
            if service_data['sflags'] > 0:
                target_team = team['id']
            if service_data['flags'] > 0:
                source_team = team['id']
            
            # If both target and source teams are found, return the round number
            if target_team and source_team:
                return round, source_team, target_team
    return None

def first_flag_team(service: Enum, data: list, team_id:int):
    # Iterate over each round in the data
    for round, round_data in enumerate(data):
        team_data = round_data['scoreboard'][team_id - 1]
        service_data = team_data['services'][service.value]
        if service_data['flags'] > 0:
            return round

def first_sflag_team(service: Enum, data: list, team_id:int):
    # Iterate over each round in the data
    for round, round_data in enumerate(data):
        team_data = round_data['scoreboard'][team_id - 1]
        service_data = team_data['services'][service.value]
        if service_data['sflags'] > 0:
            return round

def get_team_rank(team_id: int, round_number: int, data: list):
    round_data = data[round_number - 1]
    # Sort the teams by their score which is the sum of all their services' fp values
    sorted_teams = sorted(round_data['scoreboard'], key=lambda x: x['score'], reverse=True)
    # Get the rank of the team
    for idx, team in enumerate(sorted_teams):
        if team['id'] == team_id:
            return idx + 1

def get_top10_teams(round_number: int, data: list):
    round_data = data[round_number - 1]
    # Sort the teams by their score which is the sum of all their services' fp values
    sorted_teams = sorted(round_data['scoreboard'], key=lambda x: x['score'], reverse=True)
    # Get the rank of the team
    top10_teams = sorted_teams[:10]
    return top10_teams

def get_time_from_round(round_number: int):
    # 24/08/2023 10:00:00
    START_TIME = datetime.datetime(2023, 8, 24, 10, 0, 0)
    print(START_TIME)

def get_time_from_round(round_number: int, format: str = "%d/%m/%Y %H:%M:%S"):
    # 24/08/2023 10:00:00
    START_TIME = datetime.datetime(2023, 8, 24, 10, 0, 0)
    ROUND_TIME = datetime.timedelta(minutes=1)
    END_TIME = START_TIME + (ROUND_TIME * round_number)
    return END_TIME.strftime(format)

def rounds_between_losing_to_attacking(service: Enum, team_id: int, data: list):
    first_flag = first_flag_team(service, data, team_id)
    first_sflag = first_sflag_team(service, data, team_id)
    if first_flag and first_sflag:
        return first_flag - first_sflag
    else:
        return 0

def detect_downtime(service: Enum, team_id: int, data: list, service_start_round: int = 0, service_end_round: int = TOTAL_ROUNDS):
    # Iterate over each round in the data
    if not service_start_round:
        service_start_round = get_service_start_round(service, data)
    downcounter = 0
    for round, round_data in enumerate(data[service_start_round:service_end_round]):
        team_data = round_data['scoreboard'][team_id - 1]
        service_data = team_data['services'][service.value]
        if service_data['status'] != Status.OK.value:
            downcounter += 1
    if downcounter == 1:
        return 0
    else:
        return downcounter

# First Bloods
for service in Service:
    result = first_flag(service, data)
    if result:
        round_number, source_team, target_team = result
        print(f"{get_service_name(service):<37}: First flag stolen in Round {get_round_number(round_number):<27} by {get_team_name(source_team)} from {get_team_name(target_team)}")

print("#" * 100)

# First Flags SHAHEEN
for service in Service:
    first_flag_team_round = first_flag_team(service, data, TEAM_ID)
    if first_flag_team_round:
        flag_price = compute_service_price_for_round(service.value, first_flag_team_round, data)
        print(f"{get_service_name(service):<37}: First flag submitted by {get_team_name(TEAM_ID)} in Round {get_round_number(first_flag_team_round)}, the flag price was {flag_price:.2f} points.")
    else:
        print(f"{get_service_name(service):<37}: No flags submitted by {get_team_name(TEAM_ID)}")

print("#" * 100)

# Service Start Rounds
for service in Service:
    service_start_round = get_service_start_round(service, data)
    if service_start_round:
        print(f"{get_service_name(service):<37}: Service started in Round {get_round_number(service_start_round)}")


# print("#" * 100)
# service = Service.PURE
# for round in range(TOTAL_ROUNDS):
#     flags_stolen = data[round]['scoreboard'][TEAM_ID - 1]['services'][service.value]['sflags']
#     flags_submitted = data[round]['scoreboard'][TEAM_ID - 1]['services'][service.value]['flags']
#     print(f"{get_service_name(service)}: Round {get_round_number(round)}: {flags_stolen} flags stolen, {flags_submitted} flags submitted")

print("#" * 100)

for service in Service:
    time_to_replicate_attack = rounds_between_losing_to_attacking(service, TEAM_ID, data)
    downtime = detect_downtime(service, TEAM_ID, data)
    if time_to_replicate_attack or downtime:
        msg1 = f"{get_service_name(service):<37}: "
        msg2 = f"{get_round_number(time_to_replicate_attack)} rounds between losing a flag and replicating the attack"
        msg3 = f"{get_round_number(downtime, Colors.RED)} rounds of downtime"
        print(f"{msg1}{msg2} | {msg3}")


PASSMGR     : First flag stolen in Round 37   by C4T BuT S4D from BOT Team #2
PLACES      : First flag stolen in Round 266  by C4T BuT S4D from 0nePadding
LOCKSTONE   : First flag stolen in Round 163  by C4T BuT S4D from BOT Team #2
GODEEPER    : First flag stolen in Round 76   by MeowMeow from FTPPK
PURE        : First flag stolen in Round 305  by pwnthem0le from l33t1m
FUNDING     : First flag stolen in Round 460  by C4T BuT S4D from BOT Team #2
KEYS        : First flag stolen in Round 1467 by ECQ-B from cyberB0ts
SPACES      : First flag stolen in Round 1559 by You're all a bunch of fucking skids from CzechCyberTeam
DOCS        : First flag stolen in Round 1523 by SKSD from l33t1m
TOKENOURCER : First flag stolen in Round 1751 by You're all a bunch of fucking skids from 7a181d3e0746f7a247380f51df6e5ebe
NOTES       : First flag stolen in Round 1751 by You're all a bunch of fucking skids from CzechCyberTeam
###############################################################################

In [18]:
# for round in range(TOTAL_ROUNDS):
#     print(f"Round #{round}, \tflags = {data[round]['scoreboard'][2]['services'][4]['flags']}\tFlag Price: {compute_service_price_for_round(4, round, data)}")

200*55*4.5

49500.0

# Create the teams csv file

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

TEAMS_URL = "https://2023.ctf.hitb.org/hitb-ctf-phuket-2023/teams"

def get_team_names():
    # Set up the Selenium WebDriver
    driver_options = webdriver.ChromeOptions()
    driver_options.add_argument('--headless')
    driver = webdriver.Chrome(options=driver_options)
    driver.get(TEAMS_URL)

    # Give the JavaScript some time to render content
    time.sleep(5)

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    table = soup.find('table')
    headers = table.find_all('th') # type: ignore
    for row in table.find_all('tr'): # type: ignore
        cells = row.find_all('td')
        if cells:
            team_id = int(cells[0].text)
            # team_name is in cells[1] inside a div with class "ant-space ant-space-horizontal ant-space-align-center"
            team_name = cells[1].findNext('div').text
            # find next span with role="img"
            is_worldwide = "Worldwide" in cells[1].findNext('div', attrs={'class': 'mobile-only'}).text
            country_flag = cells[1].findNext('span', attrs={'role': 'img'}).text
            if is_worldwide:
                country_flag = "🌎"
            yield team_id, team_name, country_flag


for team_id, team_name, country_flag in get_team_names():
    with open("teams.csv", "a+") as f:
        f.write(f"{team_id},{team_name},{country_flag}\n")

In [11]:
import datetime
def get_time_from_round(round_number: int, format: str = "%d/%m/%Y %H:%M:%S"):
    # 24/08/2023 10:00:00
    START_TIME = datetime.datetime(2023, 8, 24, 10, 0, 0)
    ROUND_TIME = datetime.timedelta(minutes=1)
    END_TIME = START_TIME + (ROUND_TIME * round_number)
    return END_TIME.strftime(format)

get_time_from_round(TOTAL_ROUNDS)

'25/08/2023 15:51:00'